In [1]:
# coding: utf-8 
import pandas as pd
import numpy as np
import gensim
import jieba
import codecs

C:\Users\哲超\Anaconda3\lib\site-packages\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
import sqlite3 as sql3
conn = sql3.connect('test.db')

设置工作路径

In [3]:
workroot="F:\\pythonfile\\Social Network\\"
sample_wenben=workroot+"textContent.txt"
full_wenben=workroot+"textContent_full.txt"

In [4]:
txt=pd.read_table(full_wenben,encoding="utf-8",header=None)

导入停词列表

In [5]:
stoplist=workroot+"stoplist.txt"
stop=pd.read_csv(stoplist,header=None,sep='tipdm',encoding='utf-8')
stop=[x[0] for x in stop.values]

C:\Users\哲超\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  from ipykernel import kernelapp as app


补充自定义的微博停词

In [6]:
stop.extend(["【","】","\u200b"," ","\xa0",'全文','http','cn','微博','?',"＂",'Of','One','##','↓','★','戳']+[str(x) for x in range(0,2019)])

In [7]:
stop=set(stop)

In [9]:
len(txt)

196613

In [10]:
txt.head()

,0,1,2
0,1750765385,1,【工程师必备的九张机器学习&深度学习代码速查表】对于初学者来讲，入门机器学习和深度学习非常困...
1,1750765385,2,20万、50万、100万的算法工程师，到底有什么区别？-@云创大数据 （分享自 @...
2,1750765385,3,【AlphaGo彻底战胜人类意味着什么？】“你必须明白，这些人中的大部分还没有准备好去拔掉他...
3,1750765385,4,【数据太大爆内存怎么办？七条解决思路】在研究、应用机器学习算法的经历中，相信大伙儿经常遇到数...
4,1750765385,5,【工程师必备的九张机器学习&深度学习代码速查表】对于初学者来讲，入门机器学习和深度学习非常困...


# 数据探索

In [11]:
txt.columns=[["userid","num_weibo","weibo"]]

In [12]:
drop_flag=np.array([True]*len(txt))
for x in range(len(txt['weibo'])):
    if type(txt["weibo"][x])!=np.str:
        drop_flag[x]=False
        continue
    if ("删除" in txt['weibo'][x])and("微博" in txt['weibo'][x]):
        drop_flag[x]=False
txt=txt[drop_flag]

In [13]:
type("ssa")==str

True

筛选每个用户最近的50条微博

In [14]:
txt_50=txt
txt_50.index=range(len(txt_50))

In [15]:
len(txt_50)

189996

In [16]:
txt_50.head()

,userid,num_weibo,weibo
0,1750765385,1,【工程师必备的九张机器学习&深度学习代码速查表】对于初学者来讲，入门机器学习和深度学习非常困...
1,1750765385,2,20万、50万、100万的算法工程师，到底有什么区别？-@云创大数据 （分享自 @...
2,1750765385,3,【AlphaGo彻底战胜人类意味着什么？】“你必须明白，这些人中的大部分还没有准备好去拔掉他...
3,1750765385,4,【数据太大爆内存怎么办？七条解决思路】在研究、应用机器学习算法的经历中，相信大伙儿经常遇到数...
4,1750765385,5,【工程师必备的九张机器学习&深度学习代码速查表】对于初学者来讲，入门机器学习和深度学习非常困...


In [17]:
temp=txt["userid"]

In [18]:
len(temp.drop_duplicates())

992

# 导入大数据专属词汇

In [19]:
c=conn.cursor()

In [20]:
c.execute('''SELECT * FROM baike1000''')

In [21]:
r=c.fetchall()

In [22]:
len(r)

27133

In [23]:
f_dashujudict=codecs.open(workroot+"dashujucihui.txt","w",'utf-8')
for x in r:
    f_dashujudict.write(x[2]+'\r\n')

In [24]:
voc_table=pd.DataFrame(r,columns=["index","index2","text","dist"]).drop(["index","index2"],axis=1)

In [25]:
voc_table[voc_table['text']=='深度学习']

,text,dist
4,深度学习,0


In [26]:
f_voc=codecs.open(workroot+"词库\\mydict.txt",'w','utf-8')

In [27]:
for x in range(len(voc_table['text'])):
    f_voc.write(voc_table['text'][x]+'\n')

# 分词和去除停词

In [28]:
jieba.load_userdict(workroot+"词库\\mydict.txt")  

Building prefix dict from the default dictionary ...
Loading model from cache c:\temp\jieba.cache
Loading model cost 2.657 seconds.
Prefix dict has been built succesfully.


In [29]:
temp=txt.ix[0,"weibo"]

In [30]:
cut_trial=jieba.cut(temp)

In [31]:
txt_50['cut']=txt_50["weibo"].apply(jieba.cut)

In [32]:
newlist=[]
for x in range(len(txt_50)):
    try:
        p=[i for i in txt_50['cut'][x] if i not in stop]
        newlist.append(p)
    except Exception as wrongthing:
        print(x,wrongthing)
        newlist.append([])

In [33]:
txt_50['cut2']=newlist

In [34]:
txt_50_for_sql=txt_50.drop(['cut'],axis=1)
txt_50_for_sql['cut2']=txt_50_for_sql['cut2'].apply(lambda x: ' '.join(x))

In [35]:
txt_50_for_sql.to_sql(name='fenci220',con=conn,if_exists='replace')

In [36]:
len(txt_50)

189996

## 每条微博作为一个文档，训练LDA模型

In [40]:
weibo_dict=gensim.corpora.Dictionary(txt_50['cut2'])

In [41]:
weibo_corpus=[weibo_dict.doc2bow(i) for i in txt_50['cut2']]

In [ ]:
weibo_lda=gensim.models.LdaModel(weibo_corpus,id2word=weibo_dict)

In [ ]:
weibo_lda.save(workroot+"weibo220.lda_model")

In [ ]:
weibo_lda.show_topics(num_topics=8,num_words=10)

## 利用LDA模型计算文本相似度

In [37]:
def cut_clean(x):
    x=jieba.cut(x)
    z=[]
    for p in x:
        if p not in stop:
            z.append(p)
    return(z)

In [241]:
documents = weibo_lda[weibo_corpus]
query=weibo_dict.doc2bow(cut_clean("机器学习学了什么"))
query_vec = weibo_lda[query]
index = gensim.similarities.MatrixSimilarity(documents)
sims = index[query_vec]

z=pd.DataFrame(sims,columns=['similarity'])
z=z.sort("similarity",ascending=False)

KeyboardInterrupt: 

In [204]:
count=0
prev=""
for i, row in z.iterrows():
    if row['similarity']<0.999:
        if txt_50.ix[i,"weibo"]!=prev:
            print(i,txt_50.ix[i,"userid"],txt_50.ix[i,"weibo"])
            prev=txt_50.ix[i,"weibo"]
            count=count+1
            if count>=5: break

43761 2128464487 杨辉三角与二项式定理:开完庭去买根冷饮，旁边一个餐饮店，看一个初中生放学后在做作业，我多瞄了几眼，发现那孩子竟然在算一个叫二项式定理的东西，他在那里死算，也就是(a＋b)两次方，三次方，四次方的死算，我问他这应该不算作业吧，他说老师今天教因式的平方，我好玩就二次，三次，四次算算看，总 ​​​ 全文
149006 2317853822 临时工人一声吼，地球也要抖三抖。临时工人本领强，敢为中华造脊梁。临时工人胆子大，刀山火海都不怕。临时工人觉悟高，有啥责任一肩挑。临时工人会表演，扮过警察和城 管。临时工人是块砖，哪里需要哪里搬。（猫扑网友） ​​​
12565 2988319282 沐浴在春光里，显得格外神清气爽，小伙伴们趁着这大好的春光，赶紧去田野感受一下吧~ ​​​
155131 1496900383 22年前的4月15日，一列从湖南开往北京的列车上，一个没有买到坐票的老农，蹲在车厢哭泣。列车员问何故，老农大哭∶“我弟弟耀邦死了！”列车员惊，遂安排卧铺。老农名胡耀福，时81岁，两年后耀福于家乡去世，有人哀挽“国中有典型，两袖清风做赤子；天下无先例，一代皇兄是农人” ​​​
16411 1565193811 #早安，各位# 人再穷不能丢掉两样东西，一个是梦想，一个是良心，同感的转！ ​​​


## 计算每个文档中的TF-IDF

In [42]:
weibo_tf_idf=gensim.models.TfidfModel(weibo_corpus)

In [43]:
weibo_corpus_tfidf=weibo_tf_idf[weibo_corpus]

In [87]:
weibo_lda_2=gensim.models.LdaModel(weibo_corpus_tfidf,id2word=weibo_dict)
weibo_lda_2.save(workroot+"weibo220tf_idf.lda_model")

In [102]:
weibo_lda_2.show_topics(num_topics=6,num_words=6)

[(94,
  '0.014*"幸运" + 0.012*"粉丝" + 0.009*"礼物" + 0.008*"抓狂" + 0.007*"操作系统" + 0.006*"链"'),
 (82,
  '0.016*"天气" + 0.011*"指数" + 0.010*"同步" + 0.010*"密码" + 0.009*"品牌" + 0.009*"错"'),
 (57,
  '0.013*"早" + 0.013*"鼓掌" + 0.013*"校园" + 0.012*"活动" + 0.009*"转发" + 0.008*"CIO"'),
 (98,
  '0.029*"信息办" + 0.013*"新浪微博" + 0.011*"本溪" + 0.010*"减肥" + 0.010*"台湾" + 0.009*"国外"'),
 (58,
  '0.038*"笑" + 0.017*"加油" + 0.015*"女人" + 0.015*"学术" + 0.011*"死" + 0.009*"结婚"'),
 (18,
  '0.014*"微信" + 0.013*"花" + 0.013*"有人" + 0.011*"日子" + 0.011*"区域" + 0.010*"号"')]

## 基于TF-IDF的文本相似度计算

In [47]:
query_TI=weibo_dict.doc2bow(cut_clean("南大的周志华老师是机器学习专家"))
query_TI_vec= weibo_tf_idf[query_TI]
index_TI = gensim.similarities.Similarity(workroot+'TEMP',weibo_corpus_tfidf,len(weibo_dict))
sims_TI = index_TI[query_TI_vec]

In [48]:
z=pd.DataFrame(sims_TI,columns=['similarity'])
z=z.sort("similarity",ascending=False)

C:\Users\哲超\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


In [49]:
count=0
prev=""
for i, row in z.iterrows():
    if row['similarity']<0.999:
        if txt_50.ix[i,"weibo"]!=prev:
            print(i,txt_50.ix[i,"userid"],txt_50.ix[i,"weibo"])
            prev=txt_50.ix[i,"weibo"]
            count=count+1
            if count>=5: break

37013 1886032305 我为什么要读周志华老师的《机器学习》，因为我想提前看出来哪些企业需要机器学习的人帮助他们解决业务上的问题，同时我也想知道哪些人具备这个能力。那周志华老师的书选择了哪些问题，介绍了什么方法呢？今天读一读。 ​​​
64843 5172229575 周志华老师 @南大周志华 点评机器学习相关会议   写在几年前 但各会议格局没有太大变化 有一定参考价值 (评论给 周志华点评 机器学习会议   | 我爱计算机 http://t.cn/RPjBv4a ) ​​​
122491 1915548291 周志华揭开机器学习本质干货|周志华揭开机器学习本质的57张PPT ​​​
17951 1936526225 #CNCC2016#上午周志华教授报告的题目是《机器学习：发展与未来》，周老师庖丁解牛一般，深入浅出的解读了深度学习，大会现场火爆，反响热烈！周志华教授的高清大图来了~~@南大周志华 [太开心][太开心][太开心] ​​​
15062 1869170057 近日邀请周志华老师为 52cs.org 写点小文章，周老师推荐了自己以前写的这篇文章给我，放在当下读这篇文章，一点不过时，希望对大家有所启迪。 周志华：机器学习的两大派别 | 我爱计算机 http://t.cn/RqMzhUW ​​​


# 如果把一个用户作为一个文本

## 数据转换

In [42]:
def f_agguser(x):
    st=''
    for i in x["weibo"]:
        st=st+i
    return st
    
txt_user_50=txt_50.groupby("userid").apply(f_agguser)

In [43]:
txt_user_50=pd.DataFrame(txt_user_50,columns=["weibo"]).reset_index()

In [44]:
txt_user_50.head()

,userid,weibo
0,104556,抢篮球场这件事，应遵循如下原则。第一这是公共场地，任何人均可使用。第二，公共场地遵循利用率为...
1,64997683,昨天，习总书记在一带一路会上，不带片纸脱稿、四十四分钟演讲，洋洋洒洒万言，彰显了中国风范，树...
2,1010513714,#一直播# 明星、网红、大V都在一直播！快来看我的直播吧，5元红包金币已经准备好，别错过几...
3,1013701227,【夜跑西湖】2016年12月09日 ​​​【水墨西湖】2016-11-20 ​​​【风景】 ...
4,1025883887,以下照片是假冒的SOHO中国网站、假冒的SOHO中国市场部员工，他们通过网站等渠道发布虚假信...


In [45]:
txt_user_50['cut']=txt_user_50["weibo"].apply(jieba.cut)

In [46]:
newlist=[]
for x in range(len(txt_user_50)):
    try:
        p=[i for i in txt_user_50['cut'][x] if i not in stop]
        newlist.append(p)
    except Exception as wrongthing:
        print(x,wrongthing)
        newlist.append([])

In [47]:
txt_user_50['cut2']=newlist

In [48]:
txt_user_50=txt_user_50.drop("cut",axis=1)

In [50]:
weibo_user_dict=gensim.corpora.Dictionary(txt_user_50['cut2'])
weibo_user_dict.filter_extremes(no_below=4, no_above=0.5, keep_n=100000) 

In [51]:
weibo_user_corpus=[weibo_user_dict.doc2bow(i) for i in txt_user_50['cut2']]

In [71]:
weibo_user_lda=gensim.models.LdaModel(weibo_user_corpus,id2word=weibo_user_dict)

In [103]:
weibo_user_lda.show_topics(num_topics=6,num_words=3)

[(26, '0.002*"机器学习" + 0.002*"BI" + 0.002*"数据分析"'),
 (46, '0.002*"华为" + 0.002*"存储" + 0.002*"机器学习"'),
 (95, '0.002*"机器学习" + 0.002*"算法" + 0.002*"IBM"'),
 (86, '0.005*"云计算" + 0.002*"SAP" + 0.002*"算法"'),
 (13, '0.002*"机器学习" + 0.002*"深度学习" + 0.002*"IBM"'),
 (67, '0.003*"云计算" + 0.002*"存储" + 0.002*"华为"')]

# TF-IDF

In [260]:
weibo_user_tf_idf=gensim.models.TfidfModel(weibo_user_corpus)

In [261]:
z=[x for x in weibo_user_tf_idf[weibo_user_corpus]]

In [56]:
zz=dict()
for i in range(len(z)):
    zz[i]=pd.DataFrame(z[i],columns=['wordnum','tf-idf'])
    zz[i]['word']=zz[i]['wordnum'].map(lambda x:weibo_user_dict[x])
    zz[i].sort('tf-idf',ascending=False,inplace=True)

C:\Users\哲超\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [434]:
list_for_wordcloud=[]
for i in range(len(z)):
    list_for_wordcloud.extend(zz[i]['word'][:10])

In [465]:
zz[372].head(11)

,wordnum,tf-idf,word
1869,50670,0.145002,待用
1361,36810,0.140392,台风
2310,61366,0.134339,海宁
1075,29612,0.126773,嘉兴市
1244,33636,0.103573,老酒
2213,58848,0.101391,羊肉
26,627,0.099522,巡视
558,14964,0.097879,嘉兴
1386,37360,0.095956,刘说
1704,46043,0.094034,农村


In [461]:
txt_user_50[txt_user_50['userid']==1750659547]

,userid,weibo,cut2
372,1750659547,海盐生活垃圾焚烧发电厂项目停止海盐生活垃圾焚烧发电厂项目停止 ​​​#张三零的红包#天上掉红...,"[海盐, 生活, 垃圾焚烧, 发电厂, 项目, 停止, 海盐, 生活, 垃圾焚烧, 发电厂,..."


In [ ]:
pd.DataFrame(list_for_wordcloud).to_sql("taglist2",con=conn)

In [448]:
tf_idf_tag=pd.DataFrame(list_for_wordcloud,columns=['tag'])

In [453]:
tf_idf_tag["dummy"]=range(len(tf_idf_tag))

In [458]:
tf_idf_tag.groupby('tag').count().sort("dummy",ascending=False).to_csv(workroot+"tf_idf_标签.csv")

C:\Users\哲超\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [59]:
pd.DataFrame(list_for_wordcloud).to_csv(workroot+'tf-idf标签.csv')

In [62]:
tf_idf_table=pd.DataFrame(list_for_wordcloud,columns=["word"])

In [168]:
tf_idf_table.reset_index().groupby('word').count().sort('index',ascending=False).head()

C:\Users\哲超\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,index
word,
深度学习,48
机器学习,44
Learning,36
数据分析,32
云计算,30


In [89]:
weibo_user_corpus_tfidf=weibo_tf_idf[weibo_user_corpus]
weibo_lda_user_2=gensim.models.LdaModel(weibo_user_corpus_tfidf,id2word=weibo_user_dict)
weibo_lda_user_2.save(workroot+"weibo220tf_idf.lda_model")

## TF-IDF相似性计算和基于此的相似用户推荐

In [267]:
tempuser=weibo_user_tf_idf[weibo_user_corpus]
query_user_TI=weibo_user_dict.doc2bow(cut_clean("机器学习学了什么我看了机器学习和图像处理里的一些知识，感觉自己萌萌哒 高频交易 如果可以 NBA 湖人 哈哈哈哈哈哈我好喜欢THU的老师"))
query_user_TI_vec= weibo_user_tf_idf[query_user_TI]
index_user_TI = gensim.similarities.Similarity(workroot+'TEMP',tempuser,len(weibo_user_dict))
sims_user_TI = index_user_TI[query_user_TI_vec]

In [269]:
z=pd.DataFrame(sims_user_TI,columns=['similarity'])
z=z.sort("similarity",ascending=False)
count=0
prev=""
for i, row in z.iterrows():
    if row['similarity']<0.999:
        if txt_50.ix[i,"weibo"]!=prev:
            print(i,txt_user_50.ix[i,"userid"],txt_user_50.ix[i,"weibo"])
            prev=txt_50.ix[i,"weibo"]
            count=count+1
            if count>=2: break
                

604 2009753993 你经历过绝望吗。。。（by朱较瘦） ​​​[课程]带你玩转互联网金融行业的大数据应用 - 数据分析侠（分享自 @知乎 专栏） · 作者：@面包君-数据分析侠 [课程]带你玩转互联网金融行业的大数据应用 ​​​25岁Java工程师如何转型机器学习？ - 数据分析侠（分享自 @知乎 专栏） · 作者：@面包君-数据分析侠 25岁Java工程师如何转型机器学习？ ​​​分享单曲咖喱咖喱 (电视剧《欢乐颂2》插曲)（@网易云音乐） ​​​分享单曲Circle 'Round the Sun（@网易云音乐） ​​​分享单曲Drawn to You（@网易云音乐） ​​​分享单曲热爱终极 (电视剧《碧血剑》插曲)（@网易云音乐） ​​​【地铁惊现“灵魂”舞者，妹子你来自几次元[喵喵]】16日，在南昌地铁一号线上，一位妹子对着手机挥动自己的四肢，拥有灵魂舞者般的节奏感与陶醉，坐她旁边的乘客捂脸害羞。旋转跳跃，她不停歇[喵喵]地铁灵魂舞者 @梨视频 一手video的秒拍视频 ​​​《人民的名义》里赵东来提到一本神秘的书，书里有篇文章《天局》，讲的是与天下棋，胜天半子的故事，好奇百度了一下，一口气看完真是酣畅淋漓，豪气万丈。据说作者是矫健老先生，写的真太好了！推荐给大家！（by@圆滚滚的明月大魔王） ​ ​​​分享单曲我（@网易云音乐） ​​​分享单曲你的眼神 (国/粤语)（@网易云音乐） ​​​分享单曲一个人想着一个人(电视剧《终极一班2》片尾曲)（@网易云音乐） ​​​分享图片 ​​​分享单曲床边故事（@网易云音乐） ​​​23 岁到 35 岁该如何实现资产增值？如何不陷入结婚生孩子买房的恶性循环中？ - 回答作者: 面包君 http://t.cn/R6aX8fS (想看更多？下载 @知乎 App：知乎 ) ​​​现在赚多少钱，40岁以后才能财务自由？ - 回答作者: 面包君 http://t.cn/RqFi2uL (想看更多？下载 @知乎 App：知乎 ) ​​​分享单曲Something Just Like This（@网易云音乐） ​​​分享单曲I'll Be There (Korean Ver.)（@网易云音乐） ​​​分享单曲电影情圣：我在人民广场吃炸鸡（@网易云音乐） ​​​分享单曲Love（@网易云音乐） ​​​【视频：罗曼蒂克消亡史】（

C:\Users\哲超\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


## LDA相似性计算 

In [169]:
documents_user = weibo_user_lda[weibo_user_corpus]
query_user=weibo_user_dict.doc2bow(cut_clean("我看了机器学习和图像处理里的一些知识，感觉自己萌萌哒 哈哈哈哈哈哈我好喜欢THU的老师"))
query_user_vec = weibo_user_lda[query]
index_user = gensim.similarities.MatrixSimilarity(documents_user)
sims_user = index_user[query_user_vec]

In [186]:
z_user=pd.Series(sims_user)

In [187]:
z_user.sort(ascending=False)

C:\Users\哲超\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: sort is deprecated, use sort_values(inplace=True) for INPLACE sorting
  if __name__ == '__main__':


In [190]:
txt_user_50.ix[453]["cut2"]

['试听',
 '过后',
 '顿顿',
 '报',
 '课时',
 '早教',
 '班',
 '折后',
 '接近',
 '万元',
 '周去',
 '能玩到',
 '岁',
 '试',
 '听课',
 '老师',
 '教得',
 '牛',
 '期望值',
 '高',
 '几点',
 '锻炼',
 '社交',
 '平常',
 '只能',
 '三个',
 '大人',
 '外出',
 '公园',
 '社区',
 '医院',
 '疫苗',
 '碰到',
 '小朋友',
 '主动',
 '宝宝',
 '玩',
 '一会',
 '全',
 '文学',
 '点烟',
 'boy',
 '拍',
 '视频',
 '试听',
 '早',
 '教课',
 '顿顿',
 '配合',
 '度',
 '挺',
 '高',
 '兴趣',
 '远远',
 '超过',
 '道具',
 '平常',
 '生人',
 '机会',
 '少',
 '见到',
 '陌生人',
 '好奇心',
 '爆棚',
 '小话',
 '痨',
 '第一次',
 '吃',
 '西瓜',
 '拍',
 '视频',
 '尖叫',
 'boy',
 '拍',
 '视频',
 '公园',
 '溜达',
 '顿顿',
 '上车',
 '睡',
 '霸气',
 '睡姿',
 '睡',
 '小时',
 '过路',
 '娃',
 '睡',
 '真香',
 '醒来',
 '兴奋',
 '一路',
 '尖叫',
 '啊啊啊',
 '扰民',
 '程度',
 '直追',
 '广场',
 '舞',
 '大爷',
 '大妈',
 '偷拍',
 '发现',
 '拍',
 '视频',
 '号',
 '慢慢',
 '爬',
 '嘻嘻',
 '少',
 '顿顿',
 '拍照片',
 '视频',
 '昨天',
 '刚出生',
 '那段',
 '记录',
 '挺',
 '感慨',
 '记录',
 '顿是',
 '爬',
 '拍',
 '视频',
 '顿顿',
 '骑马',
 '仗',
 '剑',
 '走',
 '天涯',
 '耶',
 '边吸',
 '奶边',
 '顿顿',
 '刚出生',
 '照片',
 '视频',
 '总',
 '丑',
 '二哈',
 '皱巴巴',
 '秃子',
 '顿顿',
 '七个',
 '重庆',
 '老家',
 '各路',
 

# WORD2VEC训练和词的相似查询

In [113]:
class TextLoader(object):
    def __init__(self):
        pass
 
    def __iter__(self):
        txt_50["cut2"]
        counter = 0
        for line in txt_50["cut2"]:
            #print line
            segments = line
            yield  segments

In [114]:
sentences = TextLoader()
model = gensim.models.Word2Vec(sentences, workers=8)
model.save(workroot+'word2vector2.model')
print ('ok')

ok


In [142]:
model.most_similar(positive=['哈哈哈'])

[('哈哈哈哈', 0.9675931930541992),
 ('醉', 0.8914998173713684),
 ('cr', 0.8873447179794312),
 ('屎', 0.8871875405311584),
 ('卧槽', 0.8856359720230103),
 ('逗', 0.8849833011627197),
 ('屌', 0.8848685026168823),
 ('黄子华', 0.8804811239242554),
 ('搞笑', 0.8773742914199829),
 ('虐', 0.8765469789505005)]

In [140]:
model['哈哈哈']

array([ -7.35746086e-01,  -8.70557547e-01,   1.11802612e-04,
        -7.12881088e-01,   1.01871538e+00,  -4.20759946e-01,
        -5.34165919e-01,   1.64917380e-01,   1.45456076e+00,
         6.27968967e-01,  -1.86812412e-02,   3.34655285e-01,
        -1.39324982e-02,  -4.81354445e-01,  -2.27890685e-01,
        -3.09160560e-01,  -4.49073255e-01,   4.15806800e-01,
         7.56477535e-01,   1.10952663e+00,   6.18213236e-01,
         4.82354671e-01,  -6.43914044e-01,  -3.17264944e-02,
        -7.53571630e-01,  -4.41766046e-02,   1.05643713e+00,
        -1.65998328e+00,  -2.44534060e-01,   5.64901352e-01,
         9.02856290e-01,   6.18561447e-01,  -6.33239806e-01,
        -8.83857965e-01,   1.44037378e+00,  -7.76357114e-01,
         6.05452061e-01,   5.13439417e-01,   1.11652243e+00,
         8.44004810e-01,   2.79568266e-02,   4.55242813e-01,
        -3.80954772e-01,  -5.00492990e-01,  -1.90018658e-02,
        -5.50473154e-01,  -9.80658352e-01,   7.24812090e-01,
         7.53900707e-01,

In [ ]:
model.most_similar_cosmul

# DOC2VEC模型训练和相似推荐

In [50]:
def Doc_loader(data):
        for i in data["cut2"].index:
            yield gensim.models.doc2vec.TaggedDocument(data["cut2"][i],[i])

In [51]:
train_corpus = list(Doc_loader(txt_50))

In [52]:
train_corpus[0]

TaggedDocument(words=['工程师', '必备', '九张', '机器学习', '深度学习', '代码', '速查表', '初学者', '入门', '机器学习', '深度学习', '困难', '深度学习', '库', '难以', '理解', '收集', '多方', '资源', 'Github', '创建', '速查表', '库', '希望', '访问', '库', '完善', '速查表', '下岗工人', '铭玉其', '平凡', '何春银微', '想'], tags=[0])

In [53]:
len(train_corpus)

189996

In [54]:
docmodel = gensim.models.doc2vec.Doc2Vec(size=60, min_count=4, iter=50)

In [55]:
docmodel.build_vocab(train_corpus)

In [57]:
%time docmodel.train(train_corpus, total_examples=docmodel.corpus_count)

Wall time: 17min 9s


234753322

In [70]:
inferred_vector=docmodel.infer_vector(cut_clean('南大的周志华是机器学习专家'))

In [71]:
sims = docmodel.docvecs.most_similar([inferred_vector], topn=len(docmodel.docvecs))

In [72]:
for x in sims[:3]:
    print(txt_50.ix[x[0]]["weibo"],x[1])

人生苦短，我用Python http://t.cn/RSyg5Il - 数据科学家快报 ​​​ 0.8933787941932678
发布了头条文章：《河东村与河西村的故事》 河东村与河西村的故事 ​​​ 0.8919347524642944
20+ Natural Language Processing APIs http://t.cn/zRZU5yw ​​​ 0.8883953094482422


## 对微博用户认证信息的分词 

In [459]:
fulltable=pd.read_excel(workroot+"profile.xlsx")

In [460]:
z.

,UID,ID,关注,粉丝,地区,性别,性取向,感情状况,昵称,标签,简介,认证,认证信息,达人,degree,betweenness,closeness,pagerank,coreness,eigenvector
0,1654736305,/08girl,736,332034,北京 朝阳区,女,NaN,NaN,袁小靓,NaN,公众号queenieyxl，制片人、影评人、国际新媒体合作组织副理事长,自由撰稿人 微博签约自媒体,自由撰稿人 微博签约自媒体,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1689923063,/10086sh,237,315455,上海 普陀区,男,NaN,NaN,上海移动10086官方微博,NaN,为切实保护消费者合法权益，全面落实“客户为根、服务为本”的服务理念，为客户提供更加高效、实惠...,上海移动10086,上海移动10086,NaN,2.0,1.983333,0.283465,0.000264,2.0,6.497080e-18
2,2492374623,/10270602,572,23435,北京 朝阳区,男,NaN,NaN,一点晓博,NaN,没有人会等待幸福，只有去争取幸福。只有这样才会真正的幸福…,前 爱结网 网站运营经理,前 爱结网 网站运营经理,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1236239021,/111985513,786,1119,北京 海淀区,男,NaN,NaN,深泉,NaN,"Talk is cheap, show me the code. When in doubt...",NaN,NaN,摄影 电影 体育,3.0,0.000000,0.318476,0.000197,3.0,7.374800e-18
4,1818980321,/115wangpan,55,89925,广东,男,NaN,NaN,115官方微博,NaN,广东一一五科技股份有限公司（简称“115科技”）注册资本1亿元，总部位于广东省东莞市松山湖国...,广东一一五科技股份有限公司,广东一一五科技股份有限公司,NaN,16.0,1051.717154,0.372611,0.000684,9.0,3.147858e-03
5,1741445840,/11947511,1795,22545,北京 东城区,男,NaN,NaN,董焘,NaN,我们一起踏上英雄的旅程,NaN,NaN,美食 汽车 旅游,8.0,421.617396,0.369668,0.000414,6.0,2.620199e-03
6,1356770320,/122539732,917,1174,北京 海淀区,男,NaN,NaN,锡珍不瘦,NaN,产品一枚！ 做简单事，并坚持之！,NaN,NaN,NaN,4.0,62.400828,0.306183,0.000286,4.0,5.435850e-04
7,1810118225,/12306ng,1113,19046,北京,男,NaN,NaN,大学001,NaN,爱技术，爱生活,京东商城高级副总裁,京东商城高级副总裁,NaN,28.0,3085.009202,0.395939,0.001794,15.0,4.571822e-02
8,1862061195,/128081811,1638,1893,北京 朝阳区,女,NaN,NaN,圆圆圆圆月,NaN,每天穿梭于各种技术产品职位中，希望结交互联网各路英雄好汉！\(^o^)/~,NaN,NaN,NaN,1.0,0.000000,0.246251,0.000197,1.0,7.374800e-18
9,1893549044,/137508989,702,24583,浙江 杭州,男,NaN,NaN,数据分析故事,NaN,数据知行，感悟人生！最奢华的高雅，你崇拜不起；最淡雅的低调，你装不起；最波动的数据，你处理不...,NaN,NaN,美食 旅游 财经,48.0,4131.959520,0.406780,0.001183,17.0,6.259458e-02


In [ ]:
full